<a href="https://colab.research.google.com/github/djliden/numerai/blob/main/run_from_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run from Colab

NB Goal: Set up a system for cloning this package and running the model(s) on colab in a fairly hands-off way.

To authenticate, either upload your `.env` file with the necessary API keys to the numerai folder which you will clone below, or you will be prompted to copy the keys.

In [1]:
!git clone https://github.com/djliden/numerai.git
%cd numerai

Cloning into 'numerai'...
remote: Enumerating objects: 180, done.
remote: Counting objects: 100% (180/180), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 180 (delta 90), reused 128 (delta 52), pack-reused 0
Receiving objects: 100% (180/180), 139.19 KiB | 874.00 KiB/s, done.
Resolving deltas: 100% (90/90), done.
/content/numerai


In [17]:
%%capture
%%bash
pip install --upgrade fastai numerapi python-dotenv ipywidgets jupyter ipython ipython-genutils yacs

In [34]:
!python ./src/fastai_tabular.py ./src/config/experiments/expt_colab.yaml

DATA:
  BATCH_SIZE: 2048
  REFRESH: False
  SAVE_PROCESSED_TRAIN: True
EVAL:
  SAVE_PREDS: True
  SUBMIT_PREDS: True
MODEL:
  BATCHNORM_FINAL: True
  DROPOUT_P: [0, 0]
  EMBED_DROPOUT_P: 0
  GROUP: fastai_tabular
  LAYERS: [400, 400]
  NAME: submission_simple
  USE_BATCHNORM: True
  WEIGHT_DECAY: 0
  Y_RANGE: [0, 1]
RESULTS:
  
SESSION:
  NAME: Default
SYSTEM:
  DEBUG: False
  TIME: 2021_2_28_50
TRAIN:
  N_EPOCHS: 6
Loaded Numerai Public Key into Global Environment!
Loaded Numerai Secret Key into Global Environment!
The dataset has already been downloaded.
You can re-download it with refresh = True
setting up fastai dataloaders
Loading the processed training data from file

tcmalloc: large alloc 1591296000 bytes == 0x559614b64000 @  0x7f58ff10b1e7 0x7f58fcc8b46e 0x7f58fccdbc7b 0x7f58fccdc35f 0x7f58fcd7e103 0x55954d9840e4 0x55954d983de0 0x55954d9f86f5 0x55954d98569a 0x55954d9f3a45 0x55954d9f2b0e 0x55954d98577a 0x55954d9f3a45 0x55954d98569a 0x55954d9f3a45 0x55954d98569a 0x55954d9f3a45 0x

## Outstanding Issues:
- Progress bars not rendering correctly.

## Notes
- Colab Pro does not seem to have improved training times relative to Colab, but I gues that could depend on which GPU I'm assigned based on overall usage...
- Relatively small number of units per layer seems to be working best -- 400, 400 was good.
- Using batchnorm: no apparent improvement in training times.
    - With: .021 corr .808 sharpe
    - Without: (marginally faster): .016 correlation .652 sharpe (notably, running same config locally, ended up with much higher results .023 and > 1)
    - With + with final: even slower. .023 corr and 1.051 sharpe
- 256, 256 layers
    - Same time roughly
    - .023 corr .917 sharpe
- back to 400, increase y_range max to 1.2
    - .021, .941 -- so pretty much the same.
- dropout 0.5; embed_dropout 0.5, 20 epochs: .021, .722
- 6 epochs, dropout .1: .023, .822
    - dropout 0.2: .020, .758
    - dropout 0.3: .021, .784
- Bigger network: 1500, 1500, no dropout: .021, .848
    - with embed dropout only 0.5 BUT this doesn't make sense because our vars are treated as continuous so must just be randomness.
    - with embed [0.5, 0] to capture the intended effect: .021, .707.


## Ideas
- Option to add a new config at end -- "if you would like to add a new config, please enter a path"


In [27]:
!git pull

remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 19 (delta 11), reused 13 (delta 8), pack-reused 0
Unpacking objects: 100% (19/19), done.
From https://github.com/djliden/numerai
   73676ba..b9dabd8  main       -> origin/main
Updating 73676ba..b9dabd8
Fast-forward
 run_from_colab.ipynb   | 107 +++++++++++++++++++++++++++++++++++++++++++------
 src/config/config.py   |   2 +
 src/fastai_tabular.py  |  10 +++--
 src/utils/prep_data.py |   4 +-
 4 files changed, 105 insertions(+), 18 deletions(-)


In [ ]:
!zip -r /logs.zip /content/numerai/output/logs/*
from google.colab import files
files.download("/content/file.zip")